In [12]:
import numpy as np
import pandas as pd
import os
import bibtexparser

In [6]:
with open('./anthology+abstracts.bib', 'r') as bibtex_file:
    bib_database = bibtexparser.load(bibtex_file)

# Extracting entries

entries = bib_database.entries

AttributeError: module 'bibtexparser' has no attribute 'load'

In [11]:
bibtex_str = """
@comment{
    This is my example comment.
}

@ARTICLE{Cesar2013,
  author = {Jean César},
  title = {An amazing title},
  year = {2013},
  volume = {12},
  pages = {12--23},
  journal = {Nice Journal}
}
"""

In [ ]:
import bibtexparser
file_path = os.path.join("./", "anthology+abstracts.bib")
library = bibtexparser.parse_file(file_path)
entries = library.entries

In [20]:
entries[1]
# 2023.wsc-csdh.1/
# https://aclanthology.org/2023.wsc-csdh.1/

Entry(entry_type=`inproceedings`, key=`krishna-etal-2023-neural`, fields=`[Field(key=`title`, value=`Neural Approaches for Data Driven Dependency Parsing in {S}anskrit`, start_line=11), Field(key=`author`, value=`Krishna, Amrith  and
      Gupta, Ashim  and
      Garasangi, Deepak  and
      Sandhan, Jeevnesh  and
      Satuluri, Pavankumar  and
      Goyal, Pawan`, start_line=12), Field(key=`booktitle`, value=`Proceedings of the Computational {S}anskrit {\&} Digital Humanities: Selected papers presented at the 18th World {S}anskrit Conference`, start_line=18), Field(key=`month`, value=`jan`, start_line=19), Field(key=`year`, value=`2023`, start_line=20), Field(key=`address`, value=`Canberra, Australia (Online mode)`, start_line=21), Field(key=`publisher`, value=`Association for Computational Linguistics`, start_line=22), Field(key=`url`, value=`https://aclanthology.org/2023.wsc-csdh.1`, start_line=23), Field(key=`pages`, value=`1--20`, start_line=24)]`, start_line=10)

In [39]:
lst = str(entries[5]).split("`url` = ")[-1].split("`")
[ele for ele in lst if ele][0]

'https://aclanthology.org/2023.wsc-csdh.5'

### Test PDF to text

In [21]:
from PyPDF2 import PdfReader, PdfReader

# open the PDF file
reader = PdfReader('/Users/quert/Downloads/2022.aacl-main.11.pdf', 'rb')

page = reader.pages[1]
extracted_text = page.extract_text()
splitted = extracted_text.split("\n")
splitted
# find the indices of "Abstract" and "1 Introduction"
# splitted.index("Abstract")
# print(extracted_text)

['1322 Related Work',
 'Question generation frequently refers to the task',
 'of generating a natural language questions based',
 'on a text (Zhang et al., 2021). The generation can',
 'be conditioned on the manually spotted expected',
 'answer in the text (Murakhovs’ka et al., 2022; La-',
 'ban et al., 2022), whereas generating them in a free',
 'way (Duan et al., 2017), even potentially generating',
 'possible answers (Tafjord and Clark, 2021).',
 'In the ﬁeld of knowledge-based approaches, sev-',
 'eral propositions have been made for the verbaliza-',
 'tion of formal queries (in SQL, SPARQL, OWL,',
 'etc.) through rules or templates (Ngonga Ngomo',
 'et al., 2013, 2019; Kusuma et al., 2020), or in-',
 'termediate representations (Guo et al., 2019; Gan',
 'et al., 2021), leading to verbalizations with a vari-',
 'able naturalness. Using neural approaches, several',
 'contributions have been made to generate ques-',
 'tions from RDF triples (Han et al., 2022) or small',
 'KGs depicti